In [4]:
!pip freeze | grep scikit-learn

"grep" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [10]:
import pickle
import pandas as pd
import numpy as np

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

C:\Users\yanez\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\yanez\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [11]:
np.mean(y_pred)

16.191691679979066

In [17]:
df['ride_id'] = '2021/02_' + df.index.astype('str')

In [18]:
df['ride_id']

1                2021/02_1
2                2021/02_2
3                2021/02_3
4                2021/02_4
5                2021/02_5
                ...       
1037687    2021/02_1037687
1037688    2021/02_1037688
1037689    2021/02_1037689
1037690    2021/02_1037690
1037691    2021/02_1037691
Name: ride_id, Length: 990113, dtype: object

In [19]:
df['pred'] = y_pred

In [22]:
with open('output', 'wb') as output_file:
    df[['ride_id', 'pred']].to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [1]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1247 bytes to starter.py
